# Lab IS&A
## Bagattin Enrico - Alessandro Doretto

In [ ]:
# Import
import pandas as pd
import numpy as np
import glob
import seaborn as sns

# Modules
from utilities import *
from dataPreparation import *

In [ ]:
# Import data
paths = list(glob.glob("matches/20*.xlsx"))
matches = [pd.read_excel(path) for path in paths]
# TODO: Load matches based on number of past years choosen 
df = pd.concat(matches, ignore_index=True, sort=False)
df.info()

In [ ]:
df.head().T

In [ ]:
df.describe(include='all', percentiles=[]).T

# Cleaning and preparing data

## Remove Winner/Loser reference
All the column with Winner/Loser reference will be substituted by Player0/Player1. Then (as last step) to use both match outcomes for our prediction models we will duplicate each row (switching all the player features).

In [ ]:
neededCols = ['Location', 'Tournament', 'Series', 'Court', 'Surface',
       'Round', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'Comment', 'B365W', 'B365L', 'PSW', 'PSL', 'AvgW', 'AvgL']
df = df[neededCols]

df.columns = ['Location', 'Tournament', 'Series', 'Court', 'Surface',
       'Round', 'Player0', 'Player1', 'Rank0', 'Rank1', 'Pts0', 'Pts1',
       'Comment', 'B3650', 'B3651', 'PS0', 'PS1', 'Avg0', 'Avg1']

## Filling null:
* Rank: take the max rank plus one
* Pts: set default zero	
* Avg odd: take mode of matches with same (or similar) players rank
* B365, PS: fill with avg

In [ ]:
rankDefault = max(df['Rank0'].max(), df['Rank1'].max())+1
df.fillna({'Rank0': rankDefault, 'Rank1': rankDefault, 'Pts0': 0, 'Pts1': 0}, inplace=True)

nullOddsDf = df[df[['B3650', 'B3651', 'PS0', 'PS1', 'Avg0', 'Avg1']].isna().any(axis=1)]
for index, row in nullOddsDf.iterrows():
     if pd.isnull(row['Avg0']) or pd.isnull(row['Avg1']):
         Avg0, Avg1 = findOddsForRow(row, df.dropna(subset=['Avg0', 'Avg1']))
         df.at[index, 'Avg0'] = row['Avg0'] = Avg0
         df.at[index, 'Avg1'] = row['Avg1'] = Avg1
     if pd.isnull(row['B3650']):
         df.at[index, 'B3650'] = row['Avg0']
     if pd.isnull(row['B3651']):
         df.at[index, 'B3651'] = row['Avg1']
     if pd.isnull(row['PS0']):
         df.at[index, 'PS0'] = row['Avg0']
     if pd.isnull(row['PS1']):
         df.at[index, 'PS1'] = row['Avg1']

df.dropna(subset=['Avg0', 'Avg1'], inplace=True) # Drop rows that hasn't similar rank matches
df.info()


## New features
* [Elo rating](https://en.wikipedia.org/wiki/Elo_rating_system): a method for calculating the relative skill levels of players in zero-sum games
* Number of matches played during the last year
* Percentage of matches won during the last year
* Injuries: number matches in witch the player retired or walkover in the past year 
* Winning streak: current sequence of won games

In [ ]:
X = addEloRatingFeature(df)

## One hot encoding
* Location
* Tournament
* Series
* Court
* Surface
* Round
* Players

In [ ]:
X = pd.get_dummies(df)
print('Total number of columns:', len(X.columns))


# First analysis
# odd (inverse -> percentuale di vincita secondo il banco) --> Calcolo percentuale di successo

# Models
# Carefully-tuned random forest classifier
# Naive Bayes